In [ ]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,322 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,065 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 h

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-03-28 20:39:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.07MB/s    in 0.2s    

2023-03-28 20:39:17 (6.07 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final-NFL-DB").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load NFL and School_by_Conference Data into Spark DataFrames

In [ ]:
from pyspark import SparkFiles

NFL_url = "https://group3finalproject.s3.us-east-2.amazonaws.com/NFL_Clean_for_DB.csv"
spark.sparkContext.addFile(NFL_url)
NFL_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("NFL_Clean_for_DB.csv"), sep=",", header=True, inferSchema=True)
# Show NFL_df
NFL_df.show()

+----+--------------------+---+--------------+------+-----------+-----------+-------------+----------------+----------+-------------+-------+-----------------+-----+------------+----------+-----------+-----------+-----------------+--------+-------+
|Year|              Player|Age|        School|Height|     Weight|Sprint_40yd|Vertical_Jump|Bench_Press_Reps|Broad_Jump|Agility_3cone|Shuttle|             Team|Round|Overall_Pick|Draft_Year|        BMI|Player_Type|    Position_Type|Position|Drafted|
+----+--------------------+---+--------------+------+-----------+-----------+-------------+----------------+----------+-------------+-------+-----------------+-----+------------+----------+-----------+-----------+-----------------+--------+-------+
|2009|Beanie Wells\Well...| 20|      Ohio St.|1.8542| 106.594207|       4.38|        85.09|              25|    325.12|         null|   null|Arizona Cardinals|  1st|   31st pick|      2009|31.00419426|    offense|  backs_receivers|      RB|    Yes|
|200

In [ ]:
School_url = "https://group3finalproject.s3.us-east-2.amazonaws.com/Schools_by_Conference.csv"
spark.sparkContext.addFile(School_url)
School_by_Conference_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("Schools_by_Conference.csv"), sep=",", header=True, inferSchema=True)
# Show School_by_Conference_df
School_by_Conference_df.show()

+------------------+-----------------+
|            School|       Conference|
+------------------+-----------------+
|    North Carolina|              ACC|
|      Georgia Tech|              ACC|
|          Virginia|              ACC|
|        Louisville|              ACC|
|          Syracuse|              ACC|
|           Clemson|              ACC|
|       Wake Forest|              ACC|
|     Virginia Tech|              ACC|
|        Pittsburgh|              ACC|
|              Duke|              ACC|
|       Florida St.|              ACC|
|       Boston Col.|              ACC|
|        Miami (FL)|              ACC|
|North Carolina St.|              ACC|
|   Central Florida|American Athletic|
|        Cincinnati|American Athletic|
|     East Carolina|American Athletic|
|            Temple|American Athletic|
|     South Florida|American Athletic|
|               SMU|American Athletic|
+------------------+-----------------+
only showing top 20 rows



### Connect to the AWS RDS instance and write each DataFrame to its table

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<<endpoint_url>>:5432/<<initial_database>>"
config={"user":"<<username>>",
		    "password":"<<password>>",
		    "driver":"org.postgresql.Driver"}

In [ ]:
# Write School_by_Conference_df to table in RDS
School_by_Conference_df.write.jdbc(url=jdbc_url, table='school_by_conference', mode=mode, properties=config)

In [ ]:
# Write NFL_df to table in RDS
NFL_df.write.jdbc(url=jdbc_url, table='nfl', mode=mode, properties=config)